In [2]:
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader


In [3]:
img_comments_folder = Path("/Users/chengbai/ml/dataset/flickr30k_images/")
img_comments_file = img_comments_folder / "results.csv"

In [4]:
# The current `results.csv` file is using "| " to seperate 3 columns. 
# For the pd.read_csv, the `sep` here is given as a regular expression. 
df = pd.read_csv(img_comments_file, sep="\|\s")
df[:2]


/var/folders/46/4yv_0b8n161097gg93qqvb6w0000gn/T/ipykernel_29498/4076328688.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(img_comments_file, sep="\|\s")


,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young , White males are outside near many ..."


In [31]:
# Create Dataset 
class ImgCommentDataset(Dataset):
    def __init__(self, img_comments_file: Path):
        self.img_comments_file = img_comments_file
        
        # The current `results.csv` file is using "| " to seperate 3 columns. 
        # For the pd.read_csv, the `sep` here is given as a regular expression. 
        self.img_comments_df = pd.read_csv(img_comments_file, sep="\|\s")


    def __len__(self):
        return len(self.img_comments_df)
    
    def __getitem__(self, idx:int):
        row = self.img_comments_df[idx: idx+1]
        image_name = row["image_name"]
        comment_number = row["comment_number"]
        comment = row["comment"]
        return image_name, comment_number, comment


In [32]:
dataset = ImgCommentDataset(img_comments_file)
len(dataset)

/var/folders/46/4yv_0b8n161097gg93qqvb6w0000gn/T/ipykernel_29498/1778176618.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.img_comments_df = pd.read_csv(img_comments_file, sep="\|\s")


158915

In [35]:
image_name, comment_number, comment = dataset[0]
image_name

0    1000092795.jpg
Name: image_name, dtype: object